In [1]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *
from rsna_retro.self_supervised import *

Loading imports


In [2]:
torch.cuda.set_device(0)

In [3]:
dls = get_ss_data(256, splits=Meta.splits_stg1)

In [4]:
size=128
cb = SSCallback(BatchContrastiveLoss(), size=size, combined_loss=True)
learn = cnn_learner(dls, xresnet18, loss_func=get_loss(), lr=3e-3,
            opt_func=Adam, metrics=[], cbs=cb)

In [5]:
name = 'self_supervised_train_1'

In [6]:
do_fit(learn, 12, 4e-2)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_ss,ss_loss,orig_loss,time
0,0.130429,0.136163,0.915844,0.001948,0.134215,26:00
1,0.127196,0.150618,0.908829,0.001273,0.149345,25:58
2,0.125056,0.131514,0.917729,0.001204,0.130311,25:54
3,0.120441,0.142846,0.923445,0.001019,0.141827,25:42
4,0.114874,0.157958,0.918416,0.000898,0.157060,25:35
5,0.109794,0.116969,0.929302,0.000961,0.116008,25:51
6,0.106254,0.109913,0.935795,0.000824,0.109089,25:42
7,0.099729,0.116418,0.931211,0.000779,0.115639,25:58
8,0.093707,0.102029,0.939627,0.000787,0.101242,25:23
9,0.088832,0.097947,0.942186,0.000666,0.097281,25:37


In [7]:
# learn.load(f'runs/{name}-1')
size = 256
learn.dls = get_ss_data(192, splits=Meta.splits_stg1)
cb.update_size(size)
do_fit(learn, 12, 4e-3)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_ss,ss_loss,orig_loss,time
0,0.086686,0.101668,0.939614,0.000804,0.100864,53:07
1,0.085570,0.098466,0.942593,0.000839,0.097627,53:16
2,0.083046,0.090663,0.945993,0.000841,0.089823,53:42
3,0.082086,0.096205,0.941957,0.000749,0.095456,53:47
4,0.080665,0.095928,0.943892,0.000793,0.095134,53:17
5,0.078437,0.096408,0.946795,0.000691,0.095717,53:19
6,0.079086,0.093593,0.946604,0.000788,0.092804,53:42
7,0.074630,0.088576,0.946426,0.000702,0.087875,53:28
8,0.069809,0.089300,0.948183,0.000661,0.088640,54:06
9,0.065654,0.085585,0.950984,0.000638,0.084947,53:40


In [8]:
# learn.load(f'runs/{name}-2')
learn.dls = get_ss_data(96, splits=Meta.splits_stg1, img_dir=path_jpg)
cb.update_size(384)
do_fit(learn, 4, 4e-4)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_ss,ss_loss,orig_loss,time
0,0.072249,0.081524,0.952804,0.001194,0.080330,1:46:34
1,0.069544,0.079988,0.953708,0.001144,0.078844,1:47:37
2,0.068478,0.076986,0.955580,0.001123,0.075863,1:46:43
3,0.062416,0.078640,0.954867,0.001158,0.077481,1:46:43
